# Annika Carlson
## AEEM5042 - Module 4 Assignment

In [1]:
import numpy as np
import sys, os
sys.path.append(os.getcwd() + r"/..")
from IPython.display import display, Markdown, Math
from scipy.optimize import fsolve
from utils.tbm import shockRelation
from Project0.compflow import compflow

### Question 1

Set given/known values:

In [2]:
M0 = 0.8        # freestream mach number
A0_A1 = 0.92    # capture to highlight area ratio

Initial Assumptions:

In [3]:
y = 1.4        # specific heat ratio, cold

Additional assumptions are made later on.

Find:

a) Pressure coefficient, $C_p$, at the stagnation point

b) Inlet Mach number, $M_1$

c) Lip contraction ratio, $A_1/A_{th}$, for a throat Mach $M_{th}$ = 0.75, assuming $p_t/p_{t,th}$ = 1

d) Diffuser area ratio, $A_2/A_{th}$ if $M_2$ = 0.5 and $p_t/p_{t,th}$ = 0.98

e) The non-dimensional inlet additive drag, $D_{add}/\left( p_0 \cdot A_1 \right)$

#### a)

Start by getting the equation for pressure coefficient, $C_p$, in terms of freestream mach number using pressure isentropic relations.

$$
C_p = \frac{2\left(P_{t0} - P_0\right)}{\gamma \cdot P_0 \cdot M_0^2}
$$

where $P_{t0}$ is freestream total pressure, and $P_0$ is freestream static pressure. From isentropic relations:

$$
\frac{P_{t0}}{P_0} = \left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} \Rightarrow P_{t0} = P_0 \left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}}
$$

Plugging this into the $C_p$ equation for $P_{t0}$ and simplifying, we get:

$$
C_p = \frac{2\left[P_0 \left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} - P_0\right]}{\gamma \cdot P_0 \cdot M_0^2} = \frac{2 \cdot P_0\left[\left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} - 1\right]}{\gamma \cdot P_0 \cdot M_0^2}
$$

$$
\Rightarrow C_p = \frac{2 \left[\left(1 + \frac{\gamma - 1}{2}*M_0^2\right)^{\frac{\gamma}{\gamma - 1}} - 1\right]}{\gamma \cdot M_0^2} 
$$

This leaves us $C_p$ in terms of $\gamma$ and $M_0$, which we can now solve.

In [4]:
Cp = (2*((1 + ((y - 1)/2)*(M0**2))**(y/(y - 1)) - 1))/(y*(M0**2))
display(Math(r"\boxed{ C_p = %.2f }" % Cp))

<IPython.core.display.Math object>

#### b)

To find inlet Mach number, $M_1$, we can simply use the area ratio $A_0/A_1$ and $M_0$.

In [5]:
# Use fsolve to solve the area ratio equation for M1
def equation(M1):
    return (M1/M0)*((1 + ((y - 1)/2)*M0**2)/(1 + ((y - 1)/2)*M1**2))**((y + 1) / (2 * (y - 1))) - A0_A1

M1_guess = 0.5
M1 = fsolve(equation, M1_guess)[0]
display(Math(r"\boxed{ M_1 = %.2f }" % M1))

<IPython.core.display.Math object>

#### c)

To solve for lip contraction ratio, we use the area relation with $M_1$ and $M_{th}$.

Assume:

$M_{th}$ = 0.75

$P_{t1}/P_{t,th}$ = 1

In [6]:
Mth = 0.75          # throat Mach number
Pt1_Ptth = 1        # inlet to throat total pressure ratio

A1_Ath = (1/Pt1_Ptth)*(Mth/M1)*((1 + ((y - 1)/2)*M1**2)/(1 + ((y - 1)/2)*Mth**2))**((y + 1)/(2*(y - 1)))
display(Math(r"\boxed{ A_1/A_{th} = %.2f }" % A1_Ath))

<IPython.core.display.Math object>

#### d)

To solve for diffuser area ratio, we again use the area relation equation but with $M_2$ and $M_{th}$.

Assume:

$M_2$ = 0.5

$P_{t2}/P_{t,th}$ = 0.98

In [7]:
M2 = 0.5            # engine fan face Mach number
Pt2_Ptth = 0.98     # throat to engine fan face total pressure ratio

A2_Ath = (1/Pt2_Ptth)*(Mth/M2)*((1 + ((y - 1)/2)*M2**2)/(1 + ((y - 1)/2)*Mth**2))**((y + 1)/(2*(y - 1)))
display(Math(r"\boxed{ A_2/A_{th} = %.2f }" % A2_Ath))

<IPython.core.display.Math object>

#### e)

Now find nondimensional inlet additive drag using the equation with $M_0$, $M_1$, and $\gamma$.

In [8]:
Dadd_p0A1 = y*M0*(((1 + ((y - 1)/2)*(M0**2))/(1 + ((y - 1)/2)*(M1**2)))**(y/(2*(y - 1))))*((M1/M0)* \
        ((1 + ((y - 1)/2)*(M1**2))/(1 + ((y - 1)/2)*(M0**2)))**0.5 - 1) + \
        ((1 + ((y - 1)/2)*(M0**2))/(1 + ((y - 1)/2)*(M1**2)))**(y/(y - 1)) - 1

display(Math(r"\boxed{ D_{add}/p_0A_1 = %.2f }" % Dadd_p0A1))

<IPython.core.display.Math object>

### Question 2

Set given/known values:

In [9]:
M0 = 0.2            # freestream Mach number 
M2 = 0.65           # fan face Mach number
Pt2_Pt0 = 0.98      # inlet total pressure recovery

Assumptions:

In [10]:
y = 1.4         # specific heat ratio, cold

Find:

Capture-to-engine face area ratio, $A_0/A_2$

In [11]:
# solve the area ratio using Pt2/Pt0, M0 and M2

A0_A2 = Pt2_Pt0*(M2/M0)*((1 + ((y - 1)/2)*M0**2)/(1 + ((y - 1)/2)*M2**2))**((y + 1)/(2*(y - 1)))
display(Math(r"\boxed{ A_0/A_2 = %.2f }" % A0_A2))

<IPython.core.display.Math object>

### Question 3

Set given/known values:

In [12]:
M0 = 1.6            # flight Mach number
A0_A1 = 0.90        # inlet capture area ratio
A2_A1 = 1.2         # diffuser area ratio
Pt2_Pt1 = 0.95      # engine face to inlet total pressure loss

Assumptions:

$A_0$ is constant until the shock.

In [13]:
y = 1.4

Find:

a) Inlet Mach number, $M_1$

b) Inlet total pressure recovery, $\pi_d = P_{t2}/P_{t0}$

#### a)

From normal shock tables, we can find mach number after the normal shock given that $M_0 = M_x = 1.6$.

From the shock tables, $M_y = 0.67$

Now we can use the inlet capture area ratio to find $M_1$, now using $M_y$ in place of $M_0$.

In [14]:
M0y = 0.67

# Use fsolve to solve the area ratio equation for M1
def equation(M1):
    return (M1/M0y)*((1 + ((y - 1)/2)*M0y**2)/(1 + ((y - 1)/2)*M1**2))**((y + 1) / (2 * (y - 1))) - A0_A1

M1_guess = 0.67
M1 = fsolve(equation, M1_guess)[0]
display(Math(r"\boxed{ M_1 = %.2f }" % M1))

<IPython.core.display.Math object>

#### b)

In order to find $\pi_d$, we can use $P_{t2}/P_{t1}$ and $P_{t1}/P_{t0}$ to find $P_{t2}/P_{t0}$.

We know that $P_{t1}/P_{t0}$ is the same as the pressure ratio across the shock, $P_y/P_x$, which can be found from the shock tables for $M_0$ = 1.6.

From the tables, $P_y/P_x$ = 0.8952

Now, we find simply that $\pi_d = \frac{P_{t2}}{P_{t0}} = \frac{P_{t2}}{P_{t1}} \cdot \frac{P_{t1}}{P_{t0}}$

In [15]:
Pty_Ptx = 0.8952

Pt2_Pt0 = Pt2_Pt1*Pty_Ptx
display(Math(r"\boxed{ \pi_d = %.2f }" % Pt2_Pt0))

<IPython.core.display.Math object>

### Question 4

Set given/known values:

In [16]:
M0 = 2.5            # freestream Mach number
theta1 = 8.0        # first ramp angle
theta2 = 12.0       # second ramp angle

Assumptions:

In [17]:
y = 1.4     # specific heat ratio, cold

Find:

a) Total pressure recovery of the inlet shock system, $\pi_d$

b) Compare the total pressure recovery to a normal shock (only) inlet at the same Mach number

#### a)

Start by finding the shock angle, $\beta_1$, for the first oblique shock using the theta-beta-Mach relation. Then we can find the normal component of the freestream Mach number.

In [18]:
B1 = shockRelation(M=M0, theta=theta1, y=y)
M0n = M0*np.sin(np.deg2rad(B1))
display(Markdown(f"normal freestream mach number, $M_{{0n}}$ = ${M0n:.2f}$"))

normal freestream mach number, $M_{0n}$ = $1.25$

Now we can use the normal component, $M_{0n}$, and solve as a normal shock to get $M_{1n}$ and $P_{t1}/P_{t0}$ for the first oblique shock from the normal shock tables.

From the normal shock tables: $M_{1n} = 0.81$,  $P_{t1}/P_{t0} = 0.987057$

Now solve for M1, the actual Mach number after the first oblique shock.

In [19]:
M1n = 0.81
Pt1_Pt0 = 0.987057

M1 = M1n/np.sin(np.deg2rad(B1) - np.deg2rad(theta1))
display(Markdown(f"mach number after shock 1, $M_1$ = ${M1:.2f}$"))

mach number after shock 1, $M_1$ = $2.16$

Now, we can solve across the second oblique shock the same way.

In [20]:
B2 = shockRelation(M=M1, theta=theta2, y=y)
M1n = M1*np.sin(np.deg2rad(B2))
display(Markdown(f"normal mach number after shock 1, $M_{{1n}}$ = ${M1n:.2f}$"))

normal mach number after shock 1, $M_{1n}$ = $1.35$

From the normal shock tables: $M_{2n} = 0.76$,  $P_{t2}/P_{t1} = 0.969737$

In [21]:
M2n = 0.76
Pt2_Pt1 = 0.969737

M2 = M2n/np.sin(np.deg2rad(B2) - np.deg2rad(theta2))
display(Markdown(f"mach number after shock 2, $M_2$ = ${M2:.2f}$"))

mach number after shock 2, $M_2$ = $1.70$

Now solve across the final normal shock.

From the normal shock tables: $M_3 = 0.64$,  $P_{t3}/P_{t2} = 0.855721$

Lastly, solve for total pressure recovery of the system, $P_{t3}/P_{t0}$

In [22]:
Pt3_Pt2 = 0.855721
Pt3_Pt0 = Pt1_Pt0*Pt2_Pt1*Pt3_Pt2
display(Math(r"\boxed{ \pi_d = %.2f }" % Pt3_Pt0))

<IPython.core.display.Math object>

#### b)

If the inlet only had one normal shock, we would just solve for the pressure recovery using the freestream Mach number, $M_0 = 2.5$, across a normal shock.

From the normal shock tables: $M_1 = 0.51$,  $P_{ty}/P_{tx} = 0.499015$

Thus, $\pi_d = 0.50$.

This means that with a single normal shock at the inlet, there would be about a 50% pressure loss from the freestream to the fan face of the engine, compared to only about an 18% loss with the configuration of oblique and normal shocks shown for the same Mach number.

### Question 5

Set given/known values:

In [23]:
Ae_Ath = 2.4    # exit to throat area ratio
Pt = 100       # nozzle inlet pressure (kPa)

Assumptions:

The nozzle is choked at the throat $\left(M_{th} = 1\right)$, and isentropic until the shock.

In [24]:
y = 1.33       # specific heat ratio, hot

Find:

a) Ambient pressure, $P_0$ $\:$ Note: $\left(P_0 = P_y\right)$

b) Exhaust temperature, $T_y$

c) Mass flow rate through the nozzle

#### a)

Since we are assuming the nozzle is choked, we can use $A_e/A_{th} = A/A* = 2.4$ to look at the compressible flow tables for $\gamma = 1.33$ to find the supersonic solution for Mach number at the exit as well as the pressure ratio $P_e/P_{te}$.

From the compressible flow tables: $M_e = 2.34$,  $P_e/P_{te} = 0.074704$

And then we can solve for the ambient pressure, $P_0$, by first finding the pressure at the exit before the shock, $P_e$.

In [25]:
Pe_Pet = 0.074704
Pe = Pe_Pet*Pt
display(Markdown(f"Pressure at the exit, $P_e$ = $P_x$ = ${Pe:.2f}$ $kPa$"))

Pressure at the exit, $P_e$ = $P_x$ = $7.47$ $kPa$

Now we can solve for the pressure across the shock at the exit using the normal shock tables for $M_x = M_e = 2.34$.

From the normal shock tables: $M_y = 0.52$,  $P_y/P_x = 6.109483$

And now we can use the pressure ratio across the shock to find $P_y$, which is $P_0$.

In [26]:
P0_Pe = 6.109483
P0 = Pe*P0_Pe
display(Math(r"\boxed{ P_0 = %.2f \; kPa}" % P0))

<IPython.core.display.Math object>

#### b)

Assume:

Since the nozzle is isentropic until the shock, the total temperature will be constant from the throat until right before the shock. Thus, can use the compressible flow tables for $\gamma = 1.33$ at $M_{th} = 1$ to find the total temperature throughout the nozzle:

From the compressible flow tables: $T/T_t = 0.858369$

In [27]:
Tth = 350 + 273.15       # throat static temperature (K)
Tth_Ttth = 0.858369      # static to total temperature ratio at the throat
Ttth = Tth/Tth_Ttth      # throat total temperature (K)
Ttx = Ttth               # total temperature before the shock (K)

To find the exhaust temperature after the shock, $T_y$, we can start with using the compressible flow tables for $\gamma = 1.33$ at the Mach number before the shock, $M_e = M_x$.

From the compressible flow tables: $T_x/T_{tx} = 0.525355$

And from the normal shock tables for across the shock: $T_y/T_x = 1.823030$

These can now both be used to solve for $T_y$.

In [28]:
Tx_Ttx = 0.525355
Ty_Tx = 1.823030

Tx = Tx_Ttx*Ttx
Ty = Ty_Tx*Tx
display(Math(r"\boxed{ T_y = %.2f \; K}" % Ty))

<IPython.core.display.Math object>

#### c)

Assume:

$A_{th}$ = 0.25

In [29]:
Ath = 0.25      # throat area (m^2)

To find the mass flow rate, we can use the Mass Flow Parameter using the compressible flow code from project 0.

In [30]:
MFP = compflow(1.0, y)[4]
mdot = MFP*Ath*(Pt*1000)/np.sqrt(287*Ttth)
display(Math(r"\boxed{ \dot{m} = %.2f \; kg/s}" % mdot))

<IPython.core.display.Math object>

### Question 6

Set given/known values:

In [31]:
mdot = 75.0         # exhaust nozzle mass flow rate (kg/s)
P0 = 40.0           # freestream static pressure (kPa)
Pt8 = 350.0         # nozzle throat total pressure (kPa)
Tt8 = 1600.0        # nozzle throat total temperature (K)
A9_A8 = 1.8         # nozzle exit to throat area ratio
Pt9_Pt8 = 0.98      # nozzle exit to throat total pressure ratio
CD = 0.98           # discharge coefficient
R = 0.287           # gas constant (kJ/kg*K)
y = 1.33            # specific heat ratio, hot

Assumptions:

Nozzle is axisymmetric and choked at the throat $\left(P_{t8}/P_0 > PR_{crit}\right)$

Find:

All dimensions including throat diameter ($D_8$), exit diameter ($D_9$), divergence angle ($\alpha$), and length of divergent section ($L_s$).

Also find $C_{fg}$, $F_g$, and $C_V$.

First, we can calculate the effective area of the throat and use this to determine the throat and exit diameters.

In [32]:
# grab the MFP at the throat using Project 0 code
MFP8 = compflow(1.0, y)[4]/np.sqrt(R*1000)
A8e = (mdot*np.sqrt(Tt8))/(Pt8*1000*MFP8)

A8 = A8e/CD                     # actual throat area (m^2)
D8 = np.sqrt(4*A8/np.pi)        # throat diameter (m)

A9 = A8*(A9_A8)                 # actual nozzle exit area (m^2)
D9 = np.sqrt(4*A9/np.pi)        # throat diameter (m)


Now we solve the ideal and real nozzles to determine all of the necessary conditions to get $C_{fg}$, $F_g$, and $C_V$.

Ideal:

In [33]:
A_Astar_9i = A9/(CD*A8)     # sonic area ratio

# get M9i and P9i/Pt9i using comp flow tables at A/A*
M9i = 2.06
P9i_Pt9i = 0.117766

# since nozzle is isentropic, Pt9i = Pt8
P9i = Pt8*P9i_Pt9i

V9i = np.sqrt(R*1000.*Tt8)*np.sqrt(2*y/(y - 1)*(1 - (P9i/Pt8)**((y - 1)/y)))

Real:

In [34]:
A_Astar_9 = Pt9_Pt8*(A9/(CD*A8))     # sonic area ratio

# get M9 and P9/Pt9 using comp flow tables at A/A*
M9 = 2.03
P9_Pt9 = 0.123592

Pt9 = Pt8*Pt9_Pt8
P9 = Pt9*P9_Pt9

V9 = np.sqrt(R*1000.*Tt8)*np.sqrt(2*y/(y - 1)*(1 - (P9/Pt8)**((y - 1)/y)))

Now we can calculate $C_V$, $C_{fg}$ and $F_g$:

In [35]:
CV = V9/V9i
Cfg = CD*CV*np.sqrt((1 - (P9i/Pt8)**((y - 1)/y))/(1 - (P0/Pt8)**((y - 1)/y)))*(1 + (y - 1)/(2*y)*(1 - P0/P9)/((Pt9/P9)**((y - 1)/y) - 1))
Fg = mdot*V9 + (P9 - P0)*A9     # actual thrust

display(Math(r"\boxed{ C_V = %.3f}" % CV))
display(Math(r"\boxed{ C_{fg} = %.3f}" % Cfg))
display(Math(r"\boxed{ F_g = %.0f \: N}" % Fg))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Going back the dimensions, we can get convergence and divergence angles, $\theta$ and $\alpha$, using graphs from the text and calculate the length of the divergent section, $L_s$.

From Figure 10.60b: $\theta = 10\degree$ 

From Figure 10.61: $\alpha = 4\degree$ 

In [36]:
theta = 10.0
a = 4.0
Ls = ((D9/2) - (D8/2))/np.tan(np.deg2rad(a))

display(Math(r"\boxed{ D_8 = %.2f \: m}" % D8))
display(Math(r"\boxed{ D_9 = %.2f \: m}" % D9))
display(Math(r"\boxed{ \theta = %.1f\deg}" % theta))
display(Math(r"\boxed{ \alpha = %.1f\deg}" % a))
display(Math(r"\boxed{ L_s = %.2f \; m}" % Ls))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>